In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

loader_web=WebBaseLoader(web_path='https://docs.langchain.com/langsmith/home')
docs_web=loader_web.load()

document_web=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs_web)
db=FAISS.from_documents(document_web,embedding=embeddings)
retriever= db.as_retriever()


e:\GEN_AI_Practice\Simple_chat_bot_pai_unpaid\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\GEN_AI_Practice\Simple_chat_bot_pai_unpaid\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--BAAI--bge-base-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator

In [7]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,'Langsmith_search',
                      "Search for information about langsmith. For any question asked about langsmith you must use this tool")
retriever_tool.name


'Langsmith_search'

In [8]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)


In [9]:
tools= [tool,arxiv_tool,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\GEN_AI_Practice\\Simple_chat_bot_pai_unpaid\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 StructuredTool(name='Langsmith_search', description='Search for information about langsmith. For any question asked about langsmith you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x0000025B49B9ECA0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x0000025B49B

In [40]:
from dotenv import load_dotenv
load_dotenv()

from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
llm= ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [41]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
import os
client = Client(api_key=os.getenv("LANGCHAIN_API_KEY"))
prompt = client.pull_prompt("jacob/groq-test")

In [42]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer all questions.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]

In [49]:
system_message = """You are a helpful assistant. 
Use the provided tools to answer the user question.
IMPORTANT: Once you have found the answer in the tool output, you MUST provide a final answer immediately. 
Do not search for the same thing twice."""

In [50]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_message
)

In [51]:
from langchain_core.messages import HumanMessage

# Correct invocation for create_agent (LangGraph-based)
response = agent.invoke({
    "messages": [HumanMessage(content="Tell me about langsmith")]
})

# To see the final response text:
print(response['messages'][-1].content)

LangSmith is a platform that provides tools for developing, debugging, and deploying Large Language Model (LLM) applications. It offers features such as observability, evaluation, prompt engineering, deployment, and platform setup, all in one integrated workflow. LangSmith is framework agnostic, meaning it can be used with or without LangChain's open-source libraries, and it meets high standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. The platform allows users to create an account, set up a workspace, manage organizations, and deploy agents, and it provides resources for getting started, including documentation and a trust center.


In [52]:
from langchain_core.messages import HumanMessage

# Correct invocation for create_agent (LangGraph-based)
response = agent.invoke({
    "messages": [HumanMessage(content="Tell me a proper way to do research about something")]
})

# To see the final response text:
print(response['messages'][-1].content)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=Langsmith_search{"query": "research methods and techniques"}</function>'}}